In [1]:
import h2o
import numpy as np
import math
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init(nthreads=-1, strict_version_check=True)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_242"; OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-1~deb9u1-b08); OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp4b7mlkv9
  JVM stdout: /tmp/tmp4b7mlkv9/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp4b7mlkv9/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.4
H2O cluster version age:,27 days
H2O cluster name:,H2O_from_python_unknownUser_q261vc
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [2]:
import pandas as pd

In [3]:
!wget https://raw.githubusercontent.com/rajrohan/titanic-dataset/master/Titanic.txt

wget: /opt/conda/lib/libuuid.so.1: no version information available (required by wget)
--2020-03-22 15:59:13--  https://raw.githubusercontent.com/rajrohan/titanic-dataset/master/Titanic.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65388 (64K) [text/plain]
Saving to: ‘Titanic.txt’

Titanic.txt         100%[===================>]  63.86K  --.-KB/s    in 0.02s   

2020-03-22 15:59:13 (4.01 MB/s) - ‘Titanic.txt’ saved [65388/65388]



In [4]:
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = h2o.import_file(url)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
df.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,nan,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,nan,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,nan,7.925,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S
6,0,3,"Moran, Mr. James",male,nan,0,0,330877,8.4583,,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C


In [6]:
df = df.drop("Name")
df = df.drop('Sex')
df = df.drop('Ticket')
df = df.drop('Embarked')
df = df.drop('Cabin')

In [7]:
df.head()

PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
1,0,3,22,1,0,7.25
2,1,1,38,1,0,71.2833
3,1,3,26,0,0,7.925
4,1,1,35,1,0,53.1
5,0,3,35,0,0,8.05
6,0,3,nan,0,0,8.4583
7,0,1,54,0,0,51.8625
8,0,3,2,3,1,21.075
9,1,3,27,0,2,11.1333
10,1,2,14,1,0,30.0708


In [8]:
response = "Survived"
fm = df.col_names
fm.remove(response)

In [9]:
df[response] = df[response].asfactor()    

In [10]:
type(df[response])

h2o.frame.H2OFrame

In [11]:
parts = df.split_frame([.7])

In [12]:
train = parts[0]
test = parts[1]

In [13]:
train.head()

PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
2,1,1,38,1,0,71.2833
4,1,1,35,1,0,53.1
6,0,3,nan,0,0,8.4583
7,0,1,54,0,0,51.8625
8,0,3,2,3,1,21.075
9,1,3,27,0,2,11.1333
10,1,2,14,1,0,30.0708
11,1,3,4,1,1,16.7
12,1,1,58,0,0,26.55
13,0,3,20,0,0,8.05


In [14]:
test.head()

PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
1,0,3,22,1,0,7.25
3,1,3,26,0,0,7.925
5,0,3,35,0,0,8.05
14,0,3,39,1,5,31.275
17,0,3,2,4,1,29.125
21,0,2,35,0,0,26
25,0,3,8,3,1,21.075
27,0,3,nan,0,0,7.225
29,1,3,nan,0,0,7.8792
32,1,1,nan,1,0,146.521


In [15]:
## Depth 10 is usually plenty of depth for most datasets, but you never know
#hyper_params = {'max_depth' : range(1,30,2)}
hyper_params = {'max_depth' : [4,6,8,12,16,20]} ##faster for larger datasets

#Build initial GBM Model
gbm_grid = H2OGradientBoostingEstimator(
        ## more trees is better if the learning rate is small enough 
        ## here, use "more than enough" trees - we have early stopping
        ntrees=10000,
        ## smaller learning rate is better
        ## since we have learning_rate_annealing, we can afford to start with a 
        #bigger learning rate
        learn_rate=0.05,
        ## learning rate annealing: learning_rate shrinks by 1% after every tree 
        ## (use 1.00 to disable, but then lower the learning_rate)
        learn_rate_annealing = 0.99,
        ## sample 80% of rows per tree
        sample_rate = 0.8,
        ## sample 80% of columns per split
        col_sample_rate = 0.8,
        ## fix a random number generator seed for reproducibility
        seed = 1234,
        ## score every 10 trees to make early stopping reproducible 
        #(it depends on the scoring interval)
        score_tree_interval = 10, 
        ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
        #5 consecutive scoring events
        stopping_rounds = 5,
        stopping_metric = "AUC",
        stopping_tolerance = 1e-4)

#Build grid search with previously made GBM and hyper parameters
grid = H2OGridSearch(gbm_grid,hyper_params,
                         grid_id = 'depth_grid',
                         search_criteria = {'strategy': "Cartesian"})


#Train grid search
grid.train(x=fm, 
           y=response,
           training_frame = train
)


gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [17]:
## Depth 10 is usually plenty of depth for most datasets, but you never know
#hyper_params = {'max_depth' : range(1,30,2)}
hyper_params = {'max_depth' : [4,6,8,12,16,20]} ##faster for larger datasets

#Build initial GBM Model
gbm_grid = H2OGradientBoostingEstimator(
        ## more trees is better if the learning rate is small enough 
        ## here, use "more than enough" trees - we have early stopping
        ntrees=10000,
        ## smaller learning rate is better
        ## since we have learning_rate_annealing, we can afford to start with a 
        #bigger learning rate
        learn_rate=0.05,
        ## learning rate annealing: learning_rate shrinks by 1% after every tree 
        ## (use 1.00 to disable, but then lower the learning_rate)
        learn_rate_annealing = 0.99,
        ## sample 80% of rows per tree
        sample_rate = 0.8,
        ## sample 80% of columns per split
        col_sample_rate = 0.8,
        ## fix a random number generator seed for reproducibility
        seed = 1234,
        ## score every 10 trees to make early stopping reproducible 
        #(it depends on the scoring interval)
        score_tree_interval = 10, 
        ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
        #5 consecutive scoring events
        stopping_rounds = 5,
        stopping_metric = "AUC",
        stopping_tolerance = 1e-4)

#Build grid search with previously made GBM and hyper parameters
grid1 = H2OGridSearch(gbm_grid,hyper_params,
                         grid_id = 'depth_grid',
                         search_criteria = {'strategy': "Cartesian"})


#Train grid search
grid1.train(x=fm, 
           y=response,
           training_frame = train,
        validation_frame = test)


gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [18]:
print (grid1)

     max_depth            model_ids              logloss
0           20   depth_grid_model_6  0.22478621132949528
1           16   depth_grid_model_5  0.22660353115413343
2           12   depth_grid_model_4  0.23575999472012107
3            8   depth_grid_model_3  0.28048067742103744
4            6   depth_grid_model_2   0.3347370921644606
5            4   depth_grid_model_1   0.4121665941702506
6            4   depth_grid_model_7   0.5845668262169529
7            6   depth_grid_model_8   0.5920809477619454
8            8   depth_grid_model_9   0.6087714301346946
9           12  depth_grid_model_10   0.6233560177998693
10          16  depth_grid_model_11   0.6236388018749673
11          20  depth_grid_model_12   0.6249446466244726



In [19]:
sorted_final_grid =grid1.get_grid(sort_by='auc',decreasing=True)

print (sorted_final_grid)

     max_depth            model_ids                 auc
0           20   depth_grid_model_6  0.9925519178523737
1           16   depth_grid_model_5  0.9922813924575218
2           12   depth_grid_model_4  0.9906409725100151
3            8   depth_grid_model_3  0.9775003453515678
4            6   depth_grid_model_2   0.955156098908689
5            4   depth_grid_model_1  0.9042167426440116
6            6   depth_grid_model_8  0.7351712614870509
7            4   depth_grid_model_7   0.731124686716792
8            8   depth_grid_model_9  0.7257727652464494
9           12  depth_grid_model_10  0.7232142857142857
10          16  depth_grid_model_11  0.7181495405179616
11          20  depth_grid_model_12  0.7166353383458647



In [20]:
best_model = h2o.get_model(grid1.sorted_metric_table()['model_ids'][0])
performance_best_model = best_model.model_performance(test)
print (performance_best_model.auc())


0.7160087719298245


In [ ]:
preds = best_model.predict(test)
preds.head()
